<a href="https://colab.research.google.com/github/iftt-jonathan/database-experiment-colabs/blob/main/embed/VectorDB_Lab_CS452_(starter).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Download datasets from kaggle

import json
import os

if not os.path.exists("lex-fridman-text-embedding-3-large-128.zip"):
  kaggle_json = {"username": "michaeltreynolds","key": "149701be742f30a8a0526762c61beea0"}
  kaggle_dir = os.path.join(os.path.expanduser("~"), ".kaggle")
  os.makedirs(kaggle_dir, exist_ok=True)
  kaggle_config_path = os.path.join(kaggle_dir, "kaggle.json")
  with open(kaggle_config_path, 'w') as f:
    json.dump(kaggle_json, f)

  !kaggle datasets download -d michaeltreynolds/lex-fridman-text-embedding-3-large-128


Dataset URL: https://www.kaggle.com/datasets/michaeltreynolds/lex-fridman-text-embedding-3-large-128
License(s): MIT
 99% 586M/594M [00:01<00:00, 260MB/s]
100% 594M/594M [00:01<00:00, 324MB/s]


In [2]:
# Unzip kaggle data

!unzip lex-fridman-text-embedding-3-large-128.zip
!unzip lex-fridman-text-embedding-3-large-128/*.zip


Archive:  lex-fridman-text-embedding-3-large-128.zip
  inflating: documents/documents/batch_request_0lw3vrQqdWbdBRurTGNMHU76.jsonl  
  inflating: documents/documents/batch_request_3GozevpriRRzieX4za9xfNmY.jsonl  
  inflating: documents/documents/batch_request_3maYxwEF1svWRpbYr10br6Wv.jsonl  
  inflating: documents/documents/batch_request_7hQA9m3pUXx22JXInjYZNAu2.jsonl  
  inflating: documents/documents/batch_request_7oR3UbWsHgESFLr5eqL3jkMD.jsonl  
  inflating: documents/documents/batch_request_CXU6VbN4SDinplgj1MpILc1u.jsonl  
  inflating: documents/documents/batch_request_Fve0NjohSf5Qe0bZtAnp8D5A.jsonl  
  inflating: documents/documents/batch_request_If8QibqlgU0XRU5FcD5zpiuL.jsonl  
  inflating: documents/documents/batch_request_KQAdZBdQHYMI2MfEMXf3ytLM.jsonl  
  inflating: documents/documents/batch_request_MFMHpnaCSkNrbgAgOxCzaLOl.jsonl  
  inflating: documents/documents/batch_request_NDF2wpJfxtprLcfkIUYE2iWQ.jsonl  
  inflating: documents/documents/batch_request_S6oh8W0n2tNadcBvYxOz

In [3]:
# Use specific libraries
!pip install datasets==2.20.0 psycopg2==2.9.9 pgcopy==1.6.0
import psycopg2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.9/384.9 kB 13.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 27.3 MB/s eta 0:00:00
  Created wheel for psycopg2: filename=psycopg2-2.9.9-cp311-cp311-linux_x86_64.whl size=508998 sha256=3f834954a6894d3004f1628557d063220b5744c7e11f134b5bed13d1d0d86640
  Stored in directory: /root/.cache/pip/wheels/ab/34/b9/78ebef1b3220b4840ee482461e738566c3c9165d2b5c914f51
Successfully built psycopg2
  Attempting uninstall: psycopg2
    Found existing installation: psycopg2 2.9.10
    Uninstalling psycopg2-2.9.10:
      Successfully uninstalled psycopg2-2.9.10
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Unin

In [2]:
# Get your own trial account at timescaledb and paste your own connection string

#TODO
CONNECTION = "<Connection string here>"

In [12]:
# Use this if you want to start over on your postgres table!

DROP_TABLE = "DROP TABLE IF EXISTS podcast, segment"
with psycopg2.connect(CONNECTION) as conn:
    cursor = conn.cursor()
    cursor.execute(DROP_TABLE)
    conn.commit() # Commit the changes


In [8]:
# Useful function that takes a pd.DataFrame and copies it directly into a table.

import pandas as pd
import io
import psycopg2

from typing import List

def fast_pg_insert(df: pd.DataFrame, connection: str, table_name: str, columns: List[str]) -> None:
    """
        Inserts data from a pandas DataFrame into a PostgreSQL table using the COPY command for fast insertion.

        Parameters:
        df (pd.DataFrame): The DataFrame containing the data to be inserted.
        connection (str): The connection string to the PostgreSQL database.
        table_name (str): The name of the target table in the PostgreSQL database.
        columns (List[str]): A list of column names in the target table that correspond to the DataFrame columns.

        Returns:
        None
    """
    conn = psycopg2.connect(connection)
    _buffer = io.StringIO()
    df.to_csv(_buffer, sep=";", index=False, header=False)
    _buffer.seek(0)
    with conn.cursor() as c:
        c.copy_from(
            file=_buffer,
            table=table_name,
            sep=";",
            columns=columns,
            null=''
        )
    conn.commit()
    conn.close()

Database Schema
We will create a database with two tables: podcast and segment:

**podcast**

- PK: id
 - The unique podcast id found in the huggingface data (i,e., TRdL6ZzWBS0  is the ID for Jed Buchwald: Isaac Newton and the Philosophy of Science | Lex Fridman Podcast #214)
- title
 - The title of podcast (ie., Jed Buchwald: Isaac Newton and the Philosophy of Science | Lex Fridman Podcast #214)

**segment**

- PK: id
 - the unique identifier for the podcast segment. This was created by concatenating the podcast idx and the segment index together (ie., "0;1") is the 0th podcast and the 1st segment
This is present in the as the "custom_id" field in the `embedding.jsonl` and batch_request.jsonl files
- start_time
 - The start timestamp of the segment
- end_time
 - The end timestamp of the segment
- content
 - The raw text transcription of the podcast
- embedding
 - the 128 dimensional vector representation of the text
- FK: podcast_id
 - foreign key to podcast.id

In [ ]:
# Sample document:
# {
#   "custom_id": "89:115",
#   "url": "/v1/embeddings",
#   "method": "POST",
#   "body": {
#     "input": " have been possible without these approaches?",
#     "model": "text-embedding-3-large",
#     "dimensions": 128,
#     "metadata": {
#       "title": "Podcast: Boris Sofman: Waymo, Cozmo, Self-Driving Cars, and the Future of Robotics | Lex Fridman Podcast #241",
#       "podcast_id": "U_AREIyd0Fc",
#       "start_time": 484.52,
#       "stop_time": 487.08
#     }
#   }
# }

# Sample embedding:
# {
#   "id": "batch_req_QZBmHS7FBiVABxcsGiDx2THJ",
#   "custom_id": "89:115",
#   "response": {
#     "status_code": 200,
#     "request_id": "7a55eba082c70aca9e7872d2b694f095",
#     "body": {
#       "object": "list",
#       "data": [
#         {
#           "object": "embedding",
#           "index": 0,
#           "embedding": [
#             0.0035960325,
#             126 more lines....
#             -0.093248844
#           ]
#         }
#       ],
#       "model": "text-embedding-3-large",
#       "usage": {
#         "prompt_tokens": 7,
#         "total_tokens": 7
#       }
#     }
#   },
#   "error": null
# }

In [14]:
# Create table statements that you'll write
#TODO

# may need to run this to enable vector data type if you didn't select AI in service
CREATE_EXTENSION = "CREATE EXTENSION vector"

# TODO: Add create table statement
CREATE_PODCAST_TABLE = """
CREATE TABLE IF NOT EXISTS podcast (
  id VARCHAR(255) PRIMARY KEY,
  title TEXT
)
"""

# TODO: Add create table statement
CREATE_SEGMENT_TABLE = """
CREATE TABLE IF NOT EXISTS segment (
  id VARCHAR(255) PRIMARY KEY,
  start_time FLOAT,
  end_time FLOAT,
  content TEXT,
  embedding VECTOR(128),
  podcast_id VARCHAR(255) REFERENCES podcast(id)
)
"""

conn = psycopg2.connect(CONNECTION)
# TODO: Create tables with psycopg2 (example: https://www.geeksforgeeks.org/executing-sql-query-with-psycopg2-in-python/)
cursor = conn.cursor()

# cursor.execute(CREATE_EXTENSION)
cursor.execute(CREATE_PODCAST_TABLE)
cursor.execute(CREATE_SEGMENT_TABLE)

conn.commit()
conn.close()


In [17]:
## Extract needed data out of JSONL files. This may be the hard part!

# TODO: What data do we need? ids, titles, start and end times, text of segment, embedding vector
# TODO: What data is in the documents jsonl files?
# id(custom_id), title (metadata.title), start/end time (metadata.start_time or stop_time) text of segment (input)
# TODO: What data is in the embedding jsonl files?
# the embedding vector (body.data.embedding)
# TODO: Get some pandas data frames for our two tables so we can copy the data in!

from typing import List

documents_dir = "documents/documents"
embeddings_dir = "embedding/embedding"

documents = []
for file in os.listdir(documents_dir):
    if file.endswith(".jsonl"):
        with open(os.path.join(documents_dir, file), "r") as f:
            for line in f:
                obj = json.loads(line)
                documents.append({
                    "segment_id": obj["custom_id"],
                    "podcast_id": obj["body"]["metadata"]["podcast_id"],
                    "title": obj["body"]["metadata"]["title"],
                    "start_time": obj["body"]["metadata"]["start_time"],
                    "end_time": obj["body"]["metadata"]["stop_time"],
                    "content": obj["body"]["input"]
                })

embedding_dict = {}
for file in os.listdir(embeddings_dir):
    if file.endswith(".jsonl"):
        with open(os.path.join(embeddings_dir, file), "r") as f:
            for line in f:
                obj = json.loads(line)
                segment_id = obj["custom_id"]
                embedding = obj["response"]["body"]["data"][0]["embedding"]
                embedding_dict[segment_id] = embedding

for doc in documents:
    doc["embedding"] = embedding_dict.get(doc["segment_id"])

segment_df = pd.DataFrame(documents)
podcast_df = (
    segment_df[["podcast_id", "title"]]
    .drop_duplicates()
    .rename(columns={"podcast_id": "id"})
)

segment_df = segment_df.rename(columns={"segment_id": "id"})
segment_df = segment_df.drop(columns=["title"])

segment_df["embedding"] = segment_df["embedding"].apply(json.dumps)

print("Podcast sample:")
print(podcast_df.head())
print("\nSegment sample:")
print(segment_df.head())


# Sample document:
# {
#   "custom_id": "89:115",
#   "url": "/v1/embeddings",
#   "method": "POST",
#   "body": {
#     "input": " have been possible without these approaches?",
#     "model": "text-embedding-3-large",
#     "dimensions": 128,
#     "metadata": {
#       "title": "Podcast: Boris Sofman: Waymo, Cozmo, Self-Driving Cars, and the Future of Robotics | Lex Fridman Podcast #241",
#       "podcast_id": "U_AREIyd0Fc",
#       "start_time": 484.52,
#       "stop_time": 487.08
#     }
#   }
# }

# Sample embedding:
# {
#   "id": "batch_req_QZBmHS7FBiVABxcsGiDx2THJ",
#   "custom_id": "89:115",
#   "response": {
#     "status_code": 200,
#     "request_id": "7a55eba082c70aca9e7872d2b694f095",
#     "body": {
#       "object": "list",
#       "data": [
#         {
#           "object": "embedding",
#           "index": 0,
#           "embedding": [
#             0.0035960325,
#             126 more lines....
#             -0.093248844
#           ]
#         }
#       ],
#       "model": "text-embedding-3-large",
#       "usage": {
#         "prompt_tokens": 7,
#         "total_tokens": 7
#       }
#     }
#   },
#   "error": null
# }



Podcast sample:
               id                                              title
0     G4hL5Om4IJ4  Podcast: Jim Keller: The Future of Computing, ...
3388  zNdhgOk4-fE  Podcast: Silvio Micali: Cryptocurrency, Blockc...
4374  q7Qk4vYleXw  Podcast: Lex Fridman Podcast #100 – Alexander ...
7390  KW8Vjs84Fxg  Podcast: Ariel Ekblaw: Space Colonization and ...
9223  KMgPxVnKLSk  Podcast: Cristiano Amon: Qualcomm CEO | Lex Fr...

Segment sample:
        id   podcast_id  start_time  end_time  \
0  182:453  G4hL5Om4IJ4     1115.22   1117.14   
1  182:454  G4hL5Om4IJ4     1117.14   1121.34   
2  182:455  G4hL5Om4IJ4     1121.34   1123.14   
3  182:456  G4hL5Om4IJ4     1123.14   1125.94   
4  182:457  G4hL5Om4IJ4     1125.94   1128.10   

                                             content  \
0        Whereas Intel, when they made their mobile,   
1   their foray into mobile, they had one team do...   
2                Right? So it wasn't 10 experiments.   
3             And then their minds

In [19]:
#### Optional #####
# In addition to the embedding and document files you might like to load
# the full podcast raw data via the hugging face datasets library

# from datasets import load_dataset
# ds = load_dataset("Whispering-GPT/lex-fridman-podcast")


Before deduplication: 346
Unique podcast IDs: 346
After deduplication: 346


In [20]:
# TODO Copy all the "podcast" data into the podcast postgres table!
fast_pg_insert(podcast_df, CONNECTION, "podcast", ["id", "title"])

In [22]:
# TODO Copy all the "segment" data into the segment postgres table!
# HINT 1: use the recommender.utils.fast_pg_insert function to insert data into the database
# otherwise inserting the 800k documents will take a very, very long time
# HINT 2: if you don't want to use all your memory and crash
# colab, you'll need to either send the data up in chunks
# or write your own function for copying it up. Alternative to chunking maybe start
# with writing it to a CSV and then copy it up?
import numpy as np

def chunk_and_insert(df: pd.DataFrame, connection_string: str, table_name: str, columns: list, chunk_size: int = 5000):
    num_chunks = int(np.ceil(len(df) / chunk_size))
    for i in range(num_chunks):
        chunk_df = df.iloc[i * chunk_size: (i + 1) * chunk_size]
        fast_pg_insert(chunk_df, connection_string, table_name, columns)
        print(f"Inserted chunk {i + 1}/{num_chunks} into {table_name}")

chunk_and_insert(segment_df, CONNECTION, "segment", [
    "id", "podcast_id", "start_time", "end_time", "content", "embedding"
])

Inserted chunk 1/167 into segment
Inserted chunk 2/167 into segment
Inserted chunk 3/167 into segment
Inserted chunk 4/167 into segment
Inserted chunk 5/167 into segment
Inserted chunk 6/167 into segment
Inserted chunk 7/167 into segment
Inserted chunk 8/167 into segment
Inserted chunk 9/167 into segment
Inserted chunk 10/167 into segment
Inserted chunk 11/167 into segment
Inserted chunk 12/167 into segment
Inserted chunk 13/167 into segment
Inserted chunk 14/167 into segment
Inserted chunk 15/167 into segment
Inserted chunk 16/167 into segment
Inserted chunk 17/167 into segment
Inserted chunk 18/167 into segment
Inserted chunk 19/167 into segment
Inserted chunk 20/167 into segment
Inserted chunk 21/167 into segment
Inserted chunk 22/167 into segment
Inserted chunk 23/167 into segment
Inserted chunk 24/167 into segment
Inserted chunk 25/167 into segment
Inserted chunk 26/167 into segment
Inserted chunk 27/167 into segment
Inserted chunk 28/167 into segment
Inserted chunk 29/167 into se

In [11]:
## This script is used to query the database
import os
import psycopg2


# Write your queries
# Q1) What are the five most similar segments to segment "267:476"
# Input: "that if we were to meet alien life at some point"
# For each result return the podcast name, the segment id, segment raw text,  the start time, stop time, and embedding distance

conn = psycopg2.connect(CONNECTION)
cur = conn.cursor()
cur.execute("""
with prompt_embedding as (
  select embedding
  from segment
  where id = '267:476'
)
select podcast.title,
       segment.id,
       segment.content,
       segment.start_time,
       segment.end_time,
       segment.embedding <-> (SELECT embedding FROM prompt_embedding) as distance
from segment
join podcast on segment.podcast_id = podcast.id
where segment.id != '267:476'
order by distance
limit 5
""")
for row in cur.fetchall():
  print(row)

conn.commit()
conn.close()

('Podcast: Ryan Graves: UFOs, Fighter Jets, and Aliens | Lex Fridman Podcast #308', '113:2792', ' encounters, human beings, if we were to meet another alien', 6725.62, 6729.86, 0.6483450674336982)
('Podcast: Richard Dawkins: Evolution, Intelligence, Simulation, and Memes | Lex Fridman Podcast #87', '268:1019', ' Suppose we did meet an alien from outer space', 2900.04, 2903.0800000000004, 0.6558106859320757)
('Podcast: Jeffrey Shainline: Neuromorphic Computing and Optoelectronic Intelligence | Lex Fridman Podcast #225', '305:3600', ' but if we think of alien civilizations out there', 9479.960000000001, 9484.04, 0.6595433115268592)
('Podcast: Michio Kaku: Future of Humans, Aliens, Space Travel & Physics | Lex Fridman Podcast #45', '18:464', ' So I think when we meet alien life from outer space,', 1316.8600000000001, 1319.5800000000002, 0.6662026419636159)
('Podcast: Alien Debate: Sara Walker and Lee Cronin | Lex Fridman Podcast #279', '71:989', ' because if aliens come to us', 2342.34, 2

In [12]:
# Q2) What are the five most dissimilar segments to segment "267:476"
# Input: "that if we were to meet alien life at some point"
# For each result return the podcast name, the segment id, segment raw text, the start time, stop time, and embedding distance

conn = psycopg2.connect(CONNECTION)
cur = conn.cursor()
cur.execute("""
with prompt_embedding as (
  select embedding
  from segment
  where id = '267:476'
)
select podcast.title,
       segment.id,
       segment.content,
       segment.start_time,
       segment.end_time,
       segment.embedding <-> (SELECT embedding FROM prompt_embedding) as distance
from segment
join podcast on segment.podcast_id = podcast.id
where segment.id != '267:476'
order by distance desc
limit 5
""")
for row in cur.fetchall():
  print(row)

conn.commit()
conn.close()

('Podcast: Jason Calacanis: Startups, Angel Investing, Capitalism, and Friendship | Lex Fridman Podcast #161', '119:218', ' a 73 Mustang Grande in gold?', 519.96, 523.8000000000001, 1.6157687685840119)
('Podcast: Rana el Kaliouby: Emotion AI, Social Robots, and Self-Driving Cars | Lex Fridman Podcast #322', '133:2006', ' for 94 car models.', 5818.62, 5820.82, 1.5863359073014982)
('Podcast: Travis Stevens: Judo, Olympics, and Mental Toughness | Lex Fridman Podcast #223', '283:1488', ' when I called down to get the sauna.', 3709.34, 3711.1000000000004, 1.572552805197421)
('Podcast: Jeremy Howard: fast.ai Deep Learning Courses and Research | Lex Fridman Podcast #35', '241:1436', ' which has all the courses pre-installed.', 4068.9, 4071.1400000000003, 1.5663319710412156)
('Podcast: Joscha Bach: Nature of Reality, Dreams, and Consciousness | Lex Fridman Podcast #212', '307:3933', ' and very few are first class and some are budget.', 10648.64, 10650.960000000001, 1.5616341289820461)


In [14]:
# Q3) What are the five most similar segments to segment '48:511'

# Input: "Is it is there something especially interesting and profound to you in terms of our current deep learning neural network, artificial neural network approaches and the whatever we do understand about the biological neural network."
# For each result return the podcast name, the segment id, segment raw text,  the start time, stop time, and embedding distance


conn = psycopg2.connect(CONNECTION)
cur = conn.cursor()
cur.execute("""
with prompt_embedding as (
  select embedding
  from segment
  where id = '48:511'
)
select podcast.title,
       segment.id,
       segment.content,
       segment.start_time,
       segment.end_time,
       segment.embedding <-> (SELECT embedding FROM prompt_embedding) as distance
from segment
join podcast on segment.podcast_id = podcast.id
where segment.id != '48:511'
order by distance
limit 5
""")
for row in cur.fetchall():
  print(row)

conn.commit()
conn.close()

('Podcast: Andrew Huberman: Neuroscience of Optimal Performance | Lex Fridman Podcast #139', '155:648', ' Is there something interesting to you or fundamental to you about the circuitry of the brain', 3798.48, 3805.84, 0.652299685331962)
('Podcast: Cal Newport: Deep Work, Focus, Productivity, Email, and Social Media | Lex Fridman Podcast #166', '61:3707', ' of what we might discover about neural networks?', 8498.02, 8500.1, 0.7121050124628524)
('Podcast: Matt Botvinick: Neuroscience, Psychology, and AI at DeepMind | Lex Fridman Podcast #106', '48:512', " And our brain is there. There's some there's quite a few differences. Are some of them to you either interesting or perhaps profound in terms of in terms of the gap we might want to try to close in trying to create a human level intelligence.", 1846.84, 1865.84, 0.7195603322334674)
('Podcast: Yann LeCun: Dark Matter of Intelligence and Self-Supervised Learning | Lex Fridman Podcast #258', '276:2642', ' Have these, I mean, small pockets

In [15]:
# Q4) What are the five most similar segments to segment '51:56'

# Input: "But what about like the fundamental physics of dark energy? Is there any understanding of what the heck it is?"
# For each result return the podcast name, the segment id, segment raw text,  the start time, stop time, and embedding distance

conn = psycopg2.connect(CONNECTION)
cur = conn.cursor()
cur.execute("""
with prompt_embedding as (
  select embedding
  from segment
  where id = '51:56'
)
select podcast.title,
       segment.id,
       segment.content,
       segment.start_time,
       segment.end_time,
       segment.embedding <-> (SELECT embedding FROM prompt_embedding) as distance
from segment
join podcast on segment.podcast_id = podcast.id
where segment.id != '51:56'
order by distance
limit 5
""")
for row in cur.fetchall():
  print(row)

conn.commit()
conn.close()

('Podcast: George Hotz: Hacking the Simulation & Learning to Drive with Neural Nets | Lex Fridman Podcast #132', '308:144', " I mean, we don't understand dark energy, right?", 500.44, 502.6, 0.6681965222094363)
('Podcast: Lex Fridman: Ask Me Anything - AMA January 2021 | Lex Fridman Podcast', '243:273', " Like, what's up with this dark matter and dark energy stuff?", 946.22, 950.12, 0.7355511762966292)
('Podcast: Katherine de Kleer: Planets, Moons, Asteroids & Life in Our Solar System | Lex Fridman Podcast #184', '196:685', ' being like, what the hell is dark matter and dark energy?', 2591.72, 2595.9599999999996, 0.7631141596843518)
('Podcast: Alex Filippenko: Supernovae, Dark Energy, Aliens & the Expanding Universe | Lex Fridman Podcast #137', '51:36', ' Do we have any understanding of what the heck that thing is?', 216.0, 219.0, 0.7922019445543276)
('Podcast: Leonard Susskind: Quantum Mechanics, String Theory and Black Holes | Lex Fridman Podcast #41', '122:831', ' That is a big ques

In [24]:
# Q5) For each of the following podcast segments, find the five most similar podcast episodes. Hint: You can do this by averaging over the embedding vectors within a podcast episode.


conn = psycopg2.connect(CONNECTION)
cur = conn.cursor()

#     a) Segment "267:476"
cur.execute("""
with prompt_podcast as (
  select podcast_id
  from segment
  where id = '267:476'
), avg_prompt_embedding as (
  select avg(embedding) as avg_embedding
  from segment
  where podcast_id = (select podcast_id from prompt_podcast)
)
select podcast.title,
       avg(segment.embedding) <-> (SELECT avg_embedding FROM avg_prompt_embedding) as distance
from segment
join podcast on segment.podcast_id = podcast.id
where segment.id != '267:476'
group by podcast.id
order by distance
limit 5
""")
print("A")
for row in cur.fetchall():
  print(row)

#     b) Segment '48:511'
cur.execute("""
with prompt_embedding as (
  select embedding
  from segment
  where id = '48:511'
)
select podcast.title,
       segment.embedding <-> (SELECT embedding FROM prompt_embedding) as distance
from segment
join podcast on segment.podcast_id = podcast.id
where segment.id != '48:511'
order by distance
limit 5
""")
print("B")
for row in cur.fetchall():
  print(row)

#     c) Segment '51:56'
cur.execute("""
with prompt_embedding as (
  select embedding
  from segment
  where id = '51:56'
)
select podcast.title,
       segment.embedding <-> (SELECT embedding FROM prompt_embedding) as distance
from segment
join podcast on segment.podcast_id = podcast.id
where segment.id != '51:56'
order by distance
limit 5
""")
print("C")
for row in cur.fetchall():
  print(row)

# For each result return the Podcast title and the embedding distance

conn.commit()
conn.close()

A
('Podcast: David Silver: AlphaGo, AlphaZero, and Deep Reinforcement Learning | Lex Fridman Podcast #86', 0.0004139695643608583)
('Podcast: Demis Hassabis: DeepMind - AI, Superintelligence & the Future of Humanity | Lex Fridman Podcast #299', 0.11301193616129587)
('Podcast: Oriol Vinyals: DeepMind AlphaStar, StarCraft, and Language | Lex Fridman Podcast #20', 0.11327739413045972)
('Podcast: Ilya Sutskever: Deep Learning | Lex Fridman Podcast #94', 0.12476937431692706)
('Podcast: Greg Brockman: OpenAI and AGI | Lex Fridman Podcast #17', 0.13182197779463417)
B
('Podcast: Andrew Huberman: Neuroscience of Optimal Performance | Lex Fridman Podcast #139', 0.652299685331962)
('Podcast: Cal Newport: Deep Work, Focus, Productivity, Email, and Social Media | Lex Fridman Podcast #166', 0.7121050124628524)
('Podcast: Matt Botvinick: Neuroscience, Psychology, and AI at DeepMind | Lex Fridman Podcast #106', 0.7195603322334674)
('Podcast: Yann LeCun: Dark Matter of Intelligence and Self-Supervised L

In [ ]:
# Q6) For podcast episode id = VeH7qKZr0WI, find the five most similar podcast episodes. Hint: you can do a similar averaging procedure as Q5

# Input Episode: "Balaji Srinivasan: How to Fix Government, Twitter, Science, and the FDA | Lex Fridman Podcast #331"
# For each result return the Podcast title and the embedding distance


# Deliverables
You will turn in a ZIP or PDF file containing all your code and a PDF file with the queries and results for questions 1-7.